<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/GPAW_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run GPAW on Google-colab and calculate interface energy with jarvis-tools

In [1]:
!pip install -q condacolab jarvis-tools
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:29
🔁 Restarting kernel...


Install gpaw and other libraries from conda

In [2]:
!conda install -y -c conda-forge gpaw openmpi=4.1.2 gcc

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gpaw


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ase-3.22.1                 |     pyhd8ed1ab_1         1.6 MB  conda-forge
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py37h89c1867_0         155 KB  conda-forge
    click-8.1.3                |   py37h89c1867_0         145 KB  conda-forge
    conda-4.13.0               |   py37h89c1867_1         989 KB  c

On the prompt type 'y' and hit enter

In [4]:
!export GPAW_SETUP_PATH='/content'
!gpaw install-data .

Available setups and pseudopotentials
  [*] https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.11271.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.9672.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.8.7929.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.6.6300.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.5.3574.tar.gz

Selected gpaw-setups-0.9.20000.tar.gz.  Downloading...
Extracting tarball into .
Setups installed into /content/gpaw-setups-0.9.20000.
Register this setup path in /root/.gpaw/rc.py? [y/n] y
Setup path registered in /root/.gpaw/rc.py.
Current GPAW setup paths in order of search priority:
   1. /content/gpaw-setups-0.9.20000
Installation complete.


In [5]:
!gpaw test

 ------------------------------------------------------------------------------------------------
| python-3.7.10     /usr/local/bin/python                                                        |
| gpaw-22.1.0       /usr/local/lib/python3.7/site-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.7/site-packages/ase/                                  |
| numpy-1.21.6      /usr/local/lib/python3.7/site-packages/numpy/                                |
| scipy-1.7.3       /usr/local/lib/python3.7/site-packages/scipy/                                |
| libxc-5.2.3       yes                                                                          |
| _gpaw             /usr/local/lib/python3.7/site-packages/_gpaw.cpython-37m-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                          |
| OpenMP enabled    yes                                                                          |
| scalapack

In [6]:
!gpaw info

 ------------------------------------------------------------------------------------------------
| python-3.7.10     /usr/local/bin/python                                                        |
| gpaw-22.1.0       /usr/local/lib/python3.7/site-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.7/site-packages/ase/                                  |
| numpy-1.21.6      /usr/local/lib/python3.7/site-packages/numpy/                                |
| scipy-1.7.3       /usr/local/lib/python3.7/site-packages/scipy/                                |
| libxc-5.2.3       yes                                                                          |
| _gpaw             /usr/local/lib/python3.7/site-packages/_gpaw.cpython-37m-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                          |
| OpenMP enabled    yes                                                                          |
| scalapack

Note: click 'Runtime' and click 'Restart Runtime'

In [1]:
# Sample calculation from https://wiki.fysik.dtu.dk/gpaw/tutorialsexercises/structureoptimization/aluminium/aluminium.html
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW

name = 'Al-fcc'
a = 4.05  # fcc lattice parameter
b = a / 2

bulk = Atoms('Al',
             cell=[[0, b, b],
                   [b, 0, b],
                   [b, b, 0]],
             pbc=True)

view(bulk)

k = 4
calc = GPAW(mode=PW(300),       # cutoff
            kpts=(k, k, k),     # k-points
            txt=name + '.txt')  # output file

bulk.calc = calc

energy = bulk.get_potential_energy()
calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

Energy: -4.122151616077917 eV


A quick calculation of Al(111)-Al2O3(001) interface energy

In [8]:
import os
import numpy as np
from jarvis.analysis.interface.zur import make_interface
from jarvis.analysis.defects.surface import Surface
from gpaw import GPAW, PW

def jarvis_atoms_to_energy(atoms=None,kp=[2,2,1],xc='PBE',ecut=300):
       name='jarvis_db_gpaw'
       calc = GPAW(mode=PW(ecut),       # cutoff
            kpts=kp,     # k-points
            txt=name + '.txt')
       ase_atoms=atoms.ase_converter()
       ase_atoms.calc=calc
       atoms_en = ase_atoms.get_potential_energy()
       return atoms_en 

def get_interface_energy(
    film_atoms=None,
    subs_atoms=None,
    film_index=[1, 1, 1],
    subs_index=[0, 0, 1],
    film_thickness=25,
    subs_thickness=25,
    model_path="",
    seperation=3.0,
    vacuum=8.0,
    max_area_ratio_tol=1.00,
    max_area=500,
    ltol=0.05,
    atol=1,
    apply_strain=False,
    from_conventional_structure=True,
):
    film_surf = Surface(
        film_atoms,
        indices=film_index,
        from_conventional_structure=from_conventional_structure,
        thickness=film_thickness,
        vacuum=vacuum,
    ).make_surface()
    subs_surf = Surface(
        subs_atoms,
        indices=subs_index,
        from_conventional_structure=from_conventional_structure,
        thickness=subs_thickness,
        vacuum=vacuum,
    ).make_surface()
    het = make_interface(
        film=film_surf,
        subs=subs_surf,
        seperation=seperation,
        vacuum=vacuum,
        max_area_ratio_tol=max_area_ratio_tol,
        max_area=max_area,
        ltol=ltol,
        atol=atol,
        apply_strain=apply_strain,
    )
    film_en = jarvis_atoms_to_energy(het["film_sl"])  
    print('film_en', film_en)
    subs_en = jarvis_atoms_to_energy(het["subs_sl"]) 
    print ('subs_en', subs_en)
    intf_en = jarvis_atoms_to_energy(het["interface"])
    print('intf_en', intf_en)

    m = het["interface"].lattice.matrix
    area = np.linalg.norm(np.cross(m[0], m[1]))
    intf_energy = 16 * (intf_en - subs_en - film_en) / (area)  # J/m2
    het["interface_energy"] = intf_energy
    return het




It might take more than a hour

In [10]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
atoms_al = Atoms.from_dict(
    get_jid_data(dataset="dft_3d", jid="JVASP-816")["atoms"]
)
atoms_al2o3 = Atoms.from_dict(
    get_jid_data(dataset="dft_3d", jid="JVASP-32")["atoms"]
)
intf = get_interface_energy(
    film_atoms=atoms_al, subs_atoms=atoms_al2o3, film_thickness=10, subs_thickness=10
)
print(intf["interface_energy"] )


Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
mismatch_u,mismatch_v 0.03706886202229698 0.03706805926123646


/usr/local/lib/python3.7/site-packages/jarvis/core/atoms.py:1596: RuntimeWarning: divide by zero encountered in double_scalars
  ) / bottom.lattice_mat[1][1]


film_en -140.35238830139863
subs_en -440.90161284833647
intf_en -587.6015482599141
-1.2706412219766434
